In [31]:
!*go mod edit --replace github.com/gomlx/gomlx=${HOME}/Projects/gomlx

## Downloading Dataset

The method `mag.Download()` will download the dataset to the data directory, if not yet downloaded.
It then converts the dataset to tensors, which are then available for use.

The tensor is then saved for faster access. After saved, the next call to `mag.Download()` will take 1/2s.

In [46]:
import (
    "flag"

    // . "github.com/gomlx/gomlx/graph"
    // "github.com/gomlx/gomlx/types/shapes"
    mag "github.com/gomlx/gomlx/examples/ogbnmag"
    "github.com/janpfeifer/must"
)

var (
    flagDataDir   = flag.String("data", "~/work/ogbnmag", "Directory to cache downloaded and generated dataset files.")
)

%%
start := time.Now()
must.M(mag.Download(*flagDataDir))
fmt.Printf("Elapsed: %s\n", time.Since(start))

Elapsed: 487.422136ms


## FNN model, No Graph

The first model will only use the paper features, and no relations. It serves as a baseline.